In [1]:
import heapq #El módulo heapq implementa colas de prioridad (heaps)

In [2]:
class Node:
    def __init__(self, position, parent=None, action=None, path_cost=0):
        self.position = position
        self.parent = parent
        self.action = action  # Acción que llevó a este nodo
        self.path_cost = path_cost

    def __lt__(self, other):
        return self.path_cost < other.path_cost

class Problem:
    def __init__(self, initial_state, goal_state, maze):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.maze = maze
        # Definimos las acciones posibles: (delta_x, delta_y, nombre)
        self.actions = {
            'Up': (-1, 0),
            'Down': (1, 0),
            'Left': (0, -1),
            'Right': (0, 1)
        }
    
    def is_goal(self, state):
        return state == self.goal_state
    
    def get_actions(self, state):
        return list(self.actions.keys())
    
    def result(self, state, action):
        move = self.actions[action]
        return (state[0] + move[0], state[1] + move[1])
    
    def action_cost(self, state1, action, state2):
        return 1  # Todos los movimientos tienen costo 1
    
    def is_valid_position(self, pos):
        x, y = pos
        return (0 <= x < len(self.maze) and 
                0 <= y < len(self.maze[0]) and 
                self.maze[x][y] != "#")

In [3]:
def find_exit(maze):
    # Encontrar posición inicial (S) y salida (E)
    start = None
    end = None
    for i in range(len(maze)):
        for j in range(len(maze[i])):
            if maze[i][j] == "S":
                start = (i, j)
            elif maze[i][j] == "E":
                end = (i, j)
    
    if not start or not end:
        raise ValueError("Laberinto no tiene punto de inicio (S) o salida (E)")
    
    problem = Problem(start, end, maze)

    def manhattan_distance(pos, goal):
        return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])

    def get_neighbors(node):
        neighbors = []
        for action in problem.get_actions(node.position):
            new_pos = problem.result(node.position, action)
            if problem.is_valid_position(new_pos):
                new_cost = node.path_cost + problem.action_cost(node.position, action, new_pos)
                neighbors.append(Node(new_pos, node, action, new_cost))
        return neighbors

    start_node = Node(start, path_cost=0)
    frontier = []
    heapq.heappush(frontier, (manhattan_distance(start, end), start_node))
    reached = {start: start_node}

    while frontier:
        _, node = heapq.heappop(frontier)
        if problem.is_goal(node.position):
            return reconstruct_path(node)

        for neighbor in get_neighbors(node):
            if neighbor.position not in reached or neighbor.path_cost < reached[neighbor.position].path_cost:
                reached[neighbor.position] = neighbor
                heapq.heappush(frontier, (neighbor.path_cost + manhattan_distance(neighbor.position, end), neighbor))

    return None  # No se encontró salida

In [4]:
def reconstruct_path(node):
    path = []
    actions = []
    while node:
        path.append(node.position)
        if node.action:
            actions.append(node.action)
        node = node.parent
    path.reverse()
    actions.reverse()
    return path, actions

In [5]:
# Laberinto original
maze = [
    ["#", "#", "#", "#", "#", "#", "#", "#"],
    ["#", "S", "#", " ", "#", " ", "E", "#"],
    ["#", " ", " ", " ", "#", " ", " ", "#"],
    ["#", " ", "#", " ", " ", " ", "#", "#"],
    ["#", "#", "#", "#", "#", "#", "#", "#"],
    ["#", "#", "#", "#", "#", "#", "#", "#"]
]

print("Laberinto original:")
path, actions = find_exit(maze)
print("Camino a la salida (coordenadas):", path)
print("Acciones tomadas:", actions)

# Laberinto más grande con múltiples salidas
big_maze = [
    ["#", "#", "#", "#", "#", "#", "#", "#", "#", "#"],
    ["#", " ", " ", " ", "#", "E", " ", " ", " ", "#"],
    ["#", "S", "#", " ", "#", " ", "#", "#", " ", "#"],
    ["#", " ", "#", "E", " ", " ", "#", " ", " ", "#"],
    ["#", " ", "#", "#", "#", "#", "#", " ", "#", "#"],
    ["#", " ", " ", " ", " ", " ", " ", " ", " ", "#"],
    ["#", "#", "#", "#", "#", "#", "#", "#", " ", "#"],
    ["#", "#", "#", "#", "#", "#", "#", "#", "#", "#"]
]

Laberinto original:
Camino a la salida (coordenadas): [(1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (3, 4), (3, 5), (2, 5), (1, 5), (1, 6)]
Acciones tomadas: ['Down', 'Right', 'Right', 'Down', 'Right', 'Right', 'Up', 'Up', 'Right']


In [6]:
def find_all_exits(maze):
    # Encontrar todas las salidas
    exits = []
    for i in range(len(maze)):
        for j in range(len(maze[i])):
            if maze[i][j] == "E":
                exits.append((i, j))
    return exits

def find_nearest_exit(maze):
    start = None
    for i in range(len(maze)):
        for j in range(len(maze[i])):
            if maze[i][j] == "S":
                start = (i, j)
                break
        if start:
            break
    
    exits = find_all_exits(maze)
    if not exits:
        return None
    
    # Buscar el camino más corto a cualquier salida
    shortest_path = None
    shortest_actions = None
    min_length = float('inf')
    
    for exit_pos in exits:
        problem = Problem(start, exit_pos, maze)
        path, actions = find_exit(maze)  # Reutilizamos la función anterior
        if path and len(path) < min_length:
            min_length = len(path)
            shortest_path = path
            shortest_actions = actions
    
    return shortest_path, shortest_actions

print("\nLaberinto grande con múltiples salidas:")
path, actions = find_nearest_exit(big_maze)
print("Camino más corto a una salida (coordenadas):", path)
print("Acciones tomadas:", actions)


Laberinto grande con múltiples salidas:
Camino más corto a una salida (coordenadas): [(2, 1), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3)]
Acciones tomadas: ['Up', 'Right', 'Right', 'Down', 'Down']
